# Librerias y funciones

In [1]:
# Importaciones de bibliotecas necesarias
import pandas as pd  # Pandas para manipulación de datos tabulares
import json  # Módulo para trabajar con JSON
import ast  # Módulo para evaluar expresiones literales de Python
import re  # Módulo para trabajar con expresiones regulares
from textblob import TextBlob # Importa la clase TextBlob desde la biblioteca TextBlob
import nltk # Importa la biblioteca nltk (Natural Language Toolkit)
import csv # Importa el módulo csv en Python

# Habilita la recarga automática de módulos antes de ejecutar una celda
%load_ext autoreload
%autoreload 2

# Importa el módulo de advertencias y configura para ignorar todas las advertencias
import warnings
warnings.filterwarnings("ignore")

In [2]:
def verificar_tipo_datos(df):

    #Se crea un diccionario (mi_dict) con cinco claves: "nombre_campo", "tipo_datos", "no_nulos_%", "nulos_%", y "nulos". Estas claves se utilizarán para almacenar información sobre cada columna del DataFrame.
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    #Se realiza un bucle sobre todas las columnas del DataFrame df.
    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

        # Se calcula el porcentaje de valores no nulos en la columna actual (porcentaje_no_nulos).
        # La columna actual se agrega a la lista bajo la clave "nombre_campo".
        # Se obtiene el tipo de datos único en la columna actual y se agrega a la lista bajo la clave "tipo_datos".
        # Se agrega el porcentaje de valores no nulos a la lista bajo la clave "no_nulos_%".
        # Se agrega el porcentaje de valores nulos a la lista bajo la clave "nulos_%".
        # Se agrega la cantidad de valores nulos en la columna actual a la lista bajo la clave "nulos".

    # Se utiliza el diccionario mi_dict para crear un nuevo DataFrame llamado df_info.
    df_info = pd.DataFrame(mi_dict)

    # La función devuelve el DataFrame df_info que contiene información sobre cada columna, incluidos el nombre de la columna, el tipo de datos, el porcentaje de valores no nulos, el porcentaje de valores nulos y la cantidad de valores nulos.
    return df_info

# 1° Función `def developer( desarrollador : str ):`

In [3]:
# Ruta al archivo CSV
ruta_csv = '/content/drive/MyDrive/HENRY/3_Dataset_para_fastAPI/dataset_endpoint_1.csv'

# Leer el archivo CSV y convertirlo en un DataFrame
df_endpoint_1= pd.read_csv(ruta_csv)

In [4]:
# Mostrar el DataFrame
df_endpoint_1.sample(5,random_state=5)

,developer,release_anio,items_total,items_free,percentage_free
12732,The Sims Studio,2012,5,0,0.00%
13900,Whalebox Studio,2016,7,0,0.00%
9950,Puuba,2014,3,1,33.33%
2885,Cristi Militaru,2011,6,6,100.00%
4521,Fat Walrus Games,2016,1,0,0.00%


In [5]:
def developer(dataframe, desarrollador):
    # Filtrar el DataFrame por el desarrollador proporcionado
    df_desarrollador = dataframe[dataframe['developer'] == desarrollador]

    # Agrupar por año y calcular la cantidad de items y el porcentaje de contenido gratuito
    stats_por_anio = df_desarrollador.groupby('release_anio').agg({
        'items_free': 'sum',
        'items_total': 'sum',
        'percentage_free': lambda x: (pd.to_numeric(x.str.rstrip('%'), errors='coerce') / 100).mean() * 100
    }).reset_index()

    # Redondear a dos decimales
    stats_por_anio['percentage_free'] = stats_por_anio['percentage_free'].round(2)

    # Cambiar el nombre de la columna
    stats_por_anio = stats_por_anio.rename(columns={'release_anio': 'Año'})
    stats_por_anio = stats_por_anio.rename(columns={'items_total': 'Items'})
    stats_por_anio = stats_por_anio.rename(columns={'percentage_free': '% Free'})

    # Mostrar los resultados
    return stats_por_anio[['Año', 'Items', '% Free']]

In [6]:
developer(df_endpoint_1,'Valve')

,Año,Items,% Free
0,1998,1,0.00
1,1999,1,0.00
2,2000,2,0.00
3,2001,1,0.00
4,2003,1,0.00
5,2004,5,0.00
6,2005,1,100.00
7,2006,2,0.00
8,2007,4,50.00
9,2008,1,0.00


# 2° Función `def userdata( User_id : str ):`


In [7]:
# Ruta al archivo CSV
ruta_csv = '/content/drive/MyDrive/HENRY/3_Dataset_para_fastAPI/dataset_endpoint_2.csv'

# Leer el archivo CSV y convertirlo en un DataFrame
df_endpoint_2= pd.read_csv(ruta_csv)

In [8]:
# Mostrar el DataFrame
df_endpoint_2.sample(5,random_state=5)

,item_id,user_id,cantidad total gastado,percentage_true
5045922,304930,LypheLife,0.00,100.00%
1665262,267980,76561198001296435,6743.25,83.33%
4991534,238320,GamekungX,1679.16,100.00%
1937571,204300,76561198059866467,0.00,85.71%
999003,224600,76561198001398677,0.00,100.00%


In [9]:
def userdata(df, user_id):
    # Filtrar el DataFrame para obtener la información del usuario
    usuario = df[df['user_id'] == user_id]

    if usuario.empty:
        return f"El usuario {user_id} no existe en el DataFrame."

    # Obtener el primer valor de la Serie (ya que debería haber solo un usuario)
    dinero_gastado = usuario['cantidad total gastado'].iloc[0]
    porcentaje_recomendacion = float(usuario['percentage_true'].iloc[0].rstrip('%'))
    cantidad_items = usuario['item_id'].nunique()

    # Crear el diccionario de retorno
    resultado = {
        "Usuario": user_id,
        "Dinero gastado": f"${dinero_gastado:.2f} USD",
        "% de recomendación": f"{porcentaje_recomendacion:.2f}%",
        "Cantidad de items": cantidad_items
    }

    return resultado


In [10]:
userdata(df_endpoint_2,'GamekungX')

{'Usuario': 'GamekungX',
 'Dinero gastado': '$1679.16 USD',
 '% de recomendación': '100.00%',
 'Cantidad de items': 58}

# 3° Función `def UserForGenre( genero : str ):`

In [11]:
# Ruta al archivo CSV
ruta_csv = '/content/drive/MyDrive/HENRY/3_Dataset_para_fastAPI/dataset_endpoint_3.csv'

# Leer el archivo CSV y convertirlo en un DataFrame
df_endpoint_3= pd.read_csv(ruta_csv)

In [12]:
# Mostrar el DataFrame
df_endpoint_3.sample(5,random_state=5)

,item_id,playtime_forever,user_id,genres,release_anio
6741256,301520,88,alimkusing,RPG,2017
12884260,293540,380,76561198057233213,Action,2014
3366057,104900,851,SealSAURUS,Indie,2013
16985442,236090,0,kayne110,Adventure,2013
15129825,433850,45,76561198046378596,Early Access,2016


In [13]:
verificar_tipo_datos(df_endpoint_3)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,item_id,[<class 'int'>],100.0,0.0,0
1,playtime_forever,[<class 'int'>],100.0,0.0,0
2,user_id,[<class 'str'>],100.0,0.0,0
3,genres,[<class 'str'>],100.0,0.0,0
4,release_anio,[<class 'str'>],100.0,0.0,0


In [14]:
def UserForGenre(genre: str, df):
    # Convertir la columna 'release_anio' a tipo de dato entero
    df['release_anio'] = pd.to_numeric(df['release_anio'], errors='coerce', downcast='integer')

    # Filtrar el DataFrame por el género dado
    genre_df = df[df['genres'] == genre]

    # Convertir la columna 'playtime_forever' a minutos dividiendo por 60 y otra vez a hora diviendo por 60 y convertir a entero. Asumiendo que esta todo en segundos.
    genre_df['playtime_forever'] = (genre_df['playtime_forever'] / 60 / 60).astype(int)

    # Encontrar el usuario con más horas jugadas para el género dado
    max_playtime_user = genre_df.loc[genre_df['playtime_forever'].idxmax(), 'user_id']

    # Agrupar por año y sumar las horas jugadas
    yearly_playtime = genre_df.groupby('release_anio')['playtime_forever'].sum().reset_index()

    # Crear la lista de diccionarios para el retorno
    playtime_list = [{'Año': int(year), 'Horas': int(hours)} for year, hours in zip(yearly_playtime['release_anio'], yearly_playtime['playtime_forever'])]

    # Crear el diccionario de retorno
    result = {"Usuario con más horas jugadas para género " + genre: max_playtime_user, "Horas jugadas": playtime_list}

    return result

In [15]:
UserForGenre('Indie', df_endpoint_3)

{'Usuario con más horas jugadas para género Indie': 'wolop',
 'Horas jugadas': [{'Año': 1988, 'Horas': 8},
  {'Año': 1995, 'Horas': 0},
  {'Año': 1996, 'Horas': 0},
  {'Año': 1997, 'Horas': 0},
  {'Año': 1998, 'Horas': 0},
  {'Año': 1999, 'Horas': 44},
  {'Año': 2000, 'Horas': 0},
  {'Año': 2001, 'Horas': 0},
  {'Año': 2002, 'Horas': 2},
  {'Año': 2003, 'Horas': 237},
  {'Año': 2004, 'Horas': 0},
  {'Año': 2005, 'Horas': 28},
  {'Año': 2006, 'Horas': 214940},
  {'Año': 2007, 'Horas': 518},
  {'Año': 2008, 'Horas': 872},
  {'Año': 2009, 'Horas': 388},
  {'Año': 2010, 'Horas': 872},
  {'Año': 2011, 'Horas': 175137},
  {'Año': 2012, 'Horas': 49113},
  {'Año': 2013, 'Horas': 138537},
  {'Año': 2014, 'Horas': 20482},
  {'Año': 2015, 'Horas': 113817},
  {'Año': 2016, 'Horas': 44523},
  {'Año': 2017, 'Horas': 101984},
  {'Año': 2018, 'Horas': 0}]}

# 4° Función `def best_developer_year( año : int ):`

In [86]:
# Ruta al archivo CSV
ruta_csv = '/content/drive/MyDrive/HENRY/3_Dataset_para_fastAPI/dataset_endpoint_4.csv'

# Leer el archivo CSV y convertirlo en un DataFrame
df_endpoint_4= pd.read_csv(ruta_csv)

In [87]:
df_endpoint_4.sample(5,random_state=5)

,release_anio,sentiment_analysis,developer
45540,2017,1,Freejam
65401,2012,2,"Ubisoft Montreal, Massive Entertainment, and U..."
47264,2017,1,Freejam
28825,2011,1,Edmund McMillen and Florian Himsl
12651,2007,2,Valve


In [77]:
def best_developer_year(dataframe, year):
    # Filtrar por el año pedido
    df_year = dataframe[dataframe['release_anio'] == year]

    # Filtrar por valores iguales a 2 en la columna sentiment analysis
    df_filtered = df_year[df_year['sentiment_analysis'] == 2]

    # Agrupar por desarrollador y sumar la columna sentiment_analysis
    df_grouped = df_filtered.groupby('developer')['sentiment_analysis'].sum().reset_index()

    # Ordenar el DataFrame por la columna sumada en orden descendente
    df_sorted = df_grouped.sort_values(by='sentiment_analysis', ascending=False)

    # Obtener los tres primeros desarrolladores y sus sumas
    top_developers = df_sorted.head(3)

    # Crear el formato de retorno
    result = [{"Puesto {}: {}".format(i+1, row['developer']): row['sentiment_analysis']} for i, (_, row) in enumerate(top_developers.iterrows())]

    return result

In [88]:
best_developer_year(df_endpoint_4, 2017)

[{'Puesto 1: Smartly Dressed Games': 2850},
 {'Puesto 2: Freejam': 1176},
 {'Puesto 3: Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC': 644}]

# 5° Función `def developer_reviews_analysis( desarrolladora : str ):`

In [117]:
# Ruta al archivo CSV
ruta_csv = '/content/drive/MyDrive/HENRY/3_Dataset_para_fastAPI/dataset_endpoint_5.csv'

# Leer el archivo CSV y convertirlo en un DataFrame
df_endpoint_5= pd.read_csv(ruta_csv)

In [118]:
df_endpoint_5.sample(5,random_state=5)

,item_id,sentiment_analysis,developer
30627,49520,2,"Gearbox Software,Aspyr (Mac &amp; Linux)"
28752,231430,2,"Relic Entertainment,Feral Interactive (Mac),Fe..."
19108,230410,2,Digital Extremes
43197,268870,2,5 Lives Studios
35423,104900,0,"Trek Industries, Inc"


In [119]:
def developer_reviews_analysis(df, desarrolladora):
    # Filtrar datos para la desarrolladora proporcionada
    filtered_data = df[df['developer'] == desarrolladora]

    # Inicializar contadores
    positive_count = 0
    negative_count = 0

    # Contar reseñas positivas y negativas
    for sentiment in filtered_data['sentiment_analysis']:
        if sentiment == 0:
            negative_count += 1
        elif sentiment == 2:
            positive_count += 1

    # Crear el diccionario de resultados con el formato deseado
    result = {desarrolladora: [f"Negative = {negative_count}", f"Positive = {positive_count}"]}

    return result

In [120]:
developer_reviews_analysis(df_endpoint_5,'Valve')

{'Valve': ['Negative = 969', 'Positive = 4817']}